In [17]:
import numpy as np 
import jax 
import jax.numpy as jnp 
import pandas as pd 
import sklearn.metrics
import copy 


In [25]:
def prepare_inputs(input_df):
    """
        Prepares the input features that will be fed into the model.

        Inputs:
            input_df: the input dataframe into the function. Should consist ONLY of input features.
        Outputs:
            Z: the input feature matrix of size NxK, where K is the number of features
    """
    # Let's identify categorical columns in a dataframe
    categorical_cols = input_df.select_dtypes(include='object').columns
    
    # Let's identify the numeric columns in the dataframe
    numeric_cols = input_df.select_dtypes(include='number').columns

    # We want to construct the input features into the model
    # We will use a numpy array that contains both numeric and categorically encoded values
    X = input_df[numeric_cols].to_numpy() # (NxK)
    
    # Now we need to z-score the numeric features so that they can lead to efficient learning
    col_means = np.mean(X, axis=0) # K
    col_stds = np.std(X, axis=0, ddof=1) # K
    
    # Z-score
    # (NxK - 
    #  1xK) 
    #  / 
    #  (1xK)
    Z = (X - col_means[None, :]) / col_stds[None, :]
    
    # Now we want to code the categorical columns using one-hot encoding
    for col in categorical_cols:
        # NxC (C is the number of unique values in the column)
        # So for origin this will be Nx3 
        dummies = pd.get_dummies( input_df[col] ).to_numpy() 
        
        # concatenate dummies matrix onto Z
        #print(Z.shape)
        #print(dummies.shape)
        Z = np.hstack((Z, dummies)) 
    
    # finally we want to add a column of ones at the start of Z
    ones_col = np.ones((Z.shape[0], 1)) # Nx1
    
    Z = np.hstack((ones_col, Z))

    return Z

def forward_fn(params, Z):
    """
        The MLP forward function.
        This is a neuron network with one hidden layer and ReLU activations.
        
        Inputs:
            params: the weight of the model
            Z: the input feature matrix, as returned by prepare_inputs (size NxK)
        Output:
            yhat: the model's predictions (size N)
    """
    # compute the inputs into the neurons
    # 1xH + Nxd @ dxH = 1xH + NxH = NxH
    hidden_inputs = params['b_hidden'][None,:] + Z @ params['W_input_hidden']
    
    # next, we apply the non-linearity to those inputs
    #hidden_activations = jnp.tanh(hidden_inputs) # NxH
    hidden_activations = jnp.maximum(0, hidden_inputs) # <-- ReLU
    
    # now we compute the output
    # 1 + NxH @ H = 1 + N = N 
    f = params['b_output'] + hidden_activations @ params['W_hidden_output']
    p = 1/(1+jnp.exp(-f)) # <-- sigmoid function (the probability of a positive)

    # so that the log doesn't blow up
    p = jnp.clip(p, 0.01, 0.99)

    return p

def predict(params, input_df):
    """
        Convienience function that prepares inputs and runs the forward function.

        Inputs:
            params: the weights of the model
            input_df: input data frame (input features only, no output column).
        Output:
            yhat: the model's predictions (size N)
    """
    Z = prepare_inputs(input_df)
    return forward_fn(params, Z)

def loss_fn(params, Z, y, penalty):
    """
        Computes the CE loss function for the model.

        Inputs:
            params: the weights of the model
            Z: the input feature matrix, as returned by prepare_inputs (size NxK)
            y: actual observations (size N)
            penalty: the penalty on L2 regularization
        Output:
            loss: CE loss
    """
    p = forward_fn(params, Z) # N

    # Log Probability of each data point, given the model (shape N)
    log_probability_of_data = y * jnp.log(p) + (1-y) * jnp.log(1-p)

    # We want to maximize the log probability of the data under the model
    # but gradient descent minimizes a loss
    # So we want to minimize the negative log probability

    # compute sum of the weights squared (L2)
    sum_weights_squared = jnp.sum(jnp.square(params['W_input_hidden'])) + \
                          jnp.sum(jnp.square(params['W_hidden_output']))

    # sum of absolute values of weights (L1)
    #sum_weights_squared = jnp.sum(jnp.abs(params['W_input_hidden'])) + \
    #                      jnp.sum(jnp.abs(params['W_hidden_output']))
    
    # regularize the loss
    loss = -jnp.mean(log_probability_of_data) + penalty * sum_weights_squared
    
    return loss 

def optimize(rng, input_df, y, learning_rate, epochs, n_hidden, penalty):
    """
        Input parameters:
            rng: the random key
            input_df: dataframe containing input columns
            y: a vector of outputs that we wish to predict
            learning_rate: how quickly we want gradient descent learning
            epochs: the number of steps of gradient descent
            n_hidden: the number of hidden units
            penalty: L2 penalty
        Output:
            params: fitted model parameters
    """

    # To make this work, we need to convert y to jax
    y = jnp.array(y)
    
    # the magic: the gradient function
    # Creates a function that can calculate the  gradient of the model
    grad_fn = jax.grad(loss_fn) 
    
    # Prepare our inputs into the linear regression
    Z = prepare_inputs(input_df) # NxK

    #
    # Initialize the parameters
    #
    params = {} # initialize an empty dictionary
    # weights from inputs to hidden neurons (dxH)
    n_inputs = Z.shape[1]
    params['W_input_hidden'] = jax.random.normal(rng, (n_inputs, n_hidden)) / jnp.sqrt(n_inputs)
    rng, _ = jax.random.split(rng)  # move to the next random key
    
    # bias of the hidden neurons is initialized to zero
    params['b_hidden'] = jnp.zeros(n_hidden)
    
    # weights from the hidden neurons to the output neuron (shape is H)
    params['W_hidden_output'] = jax.random.normal(rng, (n_hidden,)) / jnp.sqrt(n_hidden)
    rng, _ = jax.random.split(rng) # move to next random key
    
    # finally, initialize the bias of the output neuron 
    params['b_output'] = 0.0
    
    # Run gradient descent loop
    for i in range(epochs):

        # Compute the gradient of the loss function with respect
        # to all model parameters
        W_grad = grad_fn(params, Z, y, penalty)
        
        # Update the parameters
        for key in params:
            params[key] = params[key] - W_grad[key] * learning_rate

    # params is the fitted parameter values
    return params

def mlp_train_test_function(rng,
                            train_df, 
                            test_df, 
                            input_cols, 
                            output_col,
                            n_hidden,
                            penalty,
                            n_epochs):
    """
        Function that trains an MLP and tests it. Returns multiple evaluation metrics.

        Inputs:
            rng: Random number key
            train_df: training data frame
            test_df: testing data frame 
            input_cols: features to use
            output_col: output to predict
            n_hidden: number of hidden units 
            penalty: L2 penalty
            n_epochs: number of epochs to run
        Outputs:
            results: A dictionary containing accuracy, accuracy_null, auc_roc, auc_pr, auc_pr_null
    """
    # build the training input data frame
    train_input_df = train_df[input_cols]

    # build the training outputs
    y = train_df[output_col].to_numpy()
    
    # Optimize the model using gradient descent
    best_params = optimize(rng = rng,
                           input_df = train_input_df,
                           y = y,
                           learning_rate = 0.1,
                           epochs = n_epochs,
                           n_hidden = n_hidden,
                           penalty = penalty)

    # build the testing input data frame
    test_input_df = test_df[input_cols]

    # Make predictions on the test set
    yhat = predict(params = best_params, input_df = test_input_df)
    
    # Calculate error of those predictions
    ytest = test_df[output_col].to_numpy()

    # We will return multiple metrics
    yhat_hard = yhat > 0.5

    # this is our null model, and it is based only on the training set
    yhat_null = jnp.mean(y) * jnp.ones_like(ytest)
    yhat_hard_null = yhat_null > 0.5
    
    return dict(
        accuracy = sklearn.metrics.accuracy_score(ytest, yhat_hard),
        accuracy_null = sklearn.metrics.accuracy_score(ytest, yhat_hard_null),
        auc_roc = sklearn.metrics.roc_auc_score(ytest, yhat), # soft decision
        auc_pr = sklearn.metrics.average_precision_score(ytest, yhat),
        auc_pr_null = sklearn.metrics.average_precision_score(ytest, yhat_null)
    )

def factory(rng, input_cols, output_col, n_hidden, penalty, n_epochs):

    def train_test_fn(train_df, test_df):

        return mlp_train_test_function(rng,
                                  train_df = train_df,
                                  test_df = test_df,
                                  input_cols = input_cols,
                                  output_col = output_col,
                                  n_hidden=n_hidden,
                                  penalty=penalty,
                                  n_epochs = n_epochs)

    return train_test_fn

In [26]:
def cv(df, train_test_fn, folds, random_state):
    """
        Cross-validation: splits dataset into N folds, repeatedly trains on N-1 folds and test on the remaining.

        Inputs:
            df: dataframe of inputs and outputs
            train_test_fn: the training and testing function used
            folds: number of folds
            random_state: pseudo random number generator state
        Output:
            metrics: loss on each split (size N)
    
    """
    # instantiate the splitter
    kf = sklearn.model_selection.KFold(n_splits=folds, 
                                       shuffle=True, 
                                       random_state=random_state)
    
    metrics = []
    for train_index, test_index in kf.split(df):
        train_df = df.iloc[train_index]
        test_df = df.iloc[test_index]
        
        # evaluate
        metric = train_test_fn(train_df, test_df)
        metrics.append(metric)
    
    return metrics 

In [27]:
df = pd.read_csv("../data/overfitting.csv")

# identify the input column names (everything except last column)
input_cols = df.columns[:-1]

# identify the output column name (last column)
output_col = df.columns[-1]

# setup the random number seed
rng = jax.random.key(42)

# setup the training and testing function by calling the factory
train_test_fn = factory(rng,
                        input_cols,
                        output_col,
                        n_hidden = 20,
                        penalty = 0.01,
                        n_epochs = 500) # penalty of 0 means no regularization

# run cross validation
results = cv(df = df,
             train_test_fn = train_test_fn,
             folds = 5,
             random_state = 234234)
# print average results in a nice way
pd.DataFrame(results).mean()

accuracy         0.849000
accuracy_null    0.481000
auc_roc          0.930357
auc_pr           0.928479
auc_pr_null      0.498000
dtype: float64

# Early Stopping

In [28]:

def optimize(rng, input_df, y, learning_rate, epochs, n_hidden, penalty):
    """
        Input parameters:
            rng: the random key
            input_df: dataframe containing input columns
            y: a vector of outputs that we wish to predict
            learning_rate: how quickly we want gradient descent learning
            epochs: the number of steps of gradient descent
            n_hidden: the number of hidden units
            penalty: L2 penalty
        Output:
            params: fitted model parameters
    """
    print("Using the Early stopping variant")
    # For demonstration, assume the function got one extra parameter
    # which is the early stopping patience which measures
    # number of epochs to wait for validation loss to increase before giving up
    early_stopping_patience = 20 
    # proportion of data to sit aside for validation
    validation_prop = 0.2
    
    # To make this work, we need to convert y to jax
    y = jnp.array(y)
    
    # the magic: the gradient function
    # Creates a function that can calculate the  gradient of the model
    grad_fn = jax.grad(loss_fn) 
    
    # Prepare our inputs into the linear regression
    Z = prepare_inputs(input_df) # NxK

    #
    # Initialize the parameters
    #
    params = {} # initialize an empty dictionary
    # weights from inputs to hidden neurons (dxH)
    n_inputs = Z.shape[1]
    params['W_input_hidden'] = jax.random.normal(rng, (n_inputs, n_hidden)) / jnp.sqrt(n_inputs)
    rng, _ = jax.random.split(rng)  # move to the next random key
    
    # bias of the hidden neurons is initialized to zero
    params['b_hidden'] = jnp.zeros(n_hidden)
    
    # weights from the hidden neurons to the output neuron (shape is H)
    params['W_hidden_output'] = jax.random.normal(rng, (n_hidden,)) / jnp.sqrt(n_hidden)
    rng, _ = jax.random.split(rng) # move to next random key
    
    # finally, initialize the bias of the output neuron 
    params['b_output'] = 0.0

    #
    # Prepare data for early stopping
    # Objective: split Z and y so that we have 
    #  Z_opt: optimization inputs and y_opt: optimization outputs
    #  Z_valid: validation inputs and y_valid: validation outputs
    #

    # randomly permute the input data
    idx = jax.random.permutation(rng, y.shape[0])

    # figure out the number of samples to be used for optimization
    n_opt = int((1-validation_prop) * y.shape[0])

    # determine data sample indecies to be used for optimization 
    # and those for validation
    opt_idx = idx[:n_opt] # grab the first n_opt elements of idx
    valid_idx = idx[n_opt:] # grab the elements after the first n_opt elements

    # finally, split the data
    Z_opt = Z[opt_idx, :] 
    y_opt = y[opt_idx]
    Z_valid = Z[valid_idx, :]
    y_valid = y[valid_idx]

    # 
    # setup the state variables for early stopping
    #

    # how many epochs has it been since the last best validation loss?
    epochs_since_last_best = 0

    # what is the best validation loss so far?
    best_valid_loss = jnp.inf 

    # what were the parameters that correspond to the best validation loss?
    best_params = copy.deepcopy(params)
    
    # Run gradient descent loop
    for i in range(epochs):

        # Compute the gradient of the loss function with respect
        # to all model parameters
        W_grad = grad_fn(params, Z_opt, y_opt, penalty)
        
        # Update the parameters
        for key in params:
            params[key] = params[key] - W_grad[key] * learning_rate

        # Now we track validation loss and decide whether we stop or not
        valid_loss = loss_fn(params, Z_valid, y_valid, penalty)

        # is it the new best loss?
        if valid_loss < best_valid_loss:
            # yes it is
            best_valid_loss = valid_loss
            best_params = copy.deepcopy(params)
            epochs_since_last_best = 0
        else:
            # no it is not
            epochs_since_last_best += 1

            # have we ran out of patience?
            if epochs_since_last_best == early_stopping_patience:
                print(f"Stopping at epoch {i}")
                break
                
    # params is the fitted parameter values
    return best_params

df = pd.read_csv("../data/overfitting.csv")

# identify the input column names (everything except last column)
input_cols = df.columns[:-1]

# identify the output column name (last column)
output_col = df.columns[-1]

# setup the random number seed
rng = jax.random.key(42)

# setup the training and testing function by calling the factory
train_test_fn = factory(rng,
                        input_cols,
                        output_col,
                        n_hidden = 20,
                        penalty = 0.01, # early stopping + regularization
                        n_epochs = 10_000) # penalty of 0 means no regularization

# run cross validation
results = cv(df = df,
             train_test_fn = train_test_fn,
             folds = 5,
             random_state = 234234)
# print average results in a nice way
pd.DataFrame(results).mean()


Using the Early stopping variant
Stopping at epoch 1174
Using the Early stopping variant
Stopping at epoch 1800
Using the Early stopping variant
Stopping at epoch 1004
Using the Early stopping variant
Stopping at epoch 1400
Using the Early stopping variant
Stopping at epoch 1283


accuracy         0.869500
accuracy_null    0.481000
auc_roc          0.945832
auc_pr           0.943220
auc_pr_null      0.498000
dtype: float64

2
1


# Multiclass Clasification

In [29]:
pi = np.array([0.3, 0.6, 0.1])
np.argmax(pi)

np.int64(1)

In [31]:
yactual = np.array([0, 1, 2, 2, 1])
ypred =   np.array([0, 1, 2, 1, 0])
sklearn.metrics.confusion_matrix(yactual, ypred)

array([[1, 0, 0],
       [1, 1, 0],
       [0, 1, 1]])

In [32]:
sklearn.metrics.accuracy_score(yactual, ypred)

0.6

In [34]:
sklearn.metrics.balanced_accuracy_score(yactual, jnp.zeros_like(ypred))

0.3333333333333333